In [1]:
import wandb
wandb.login()

import numpy as np
from PT_files.model import DnCNN, DnCNN_B

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mdowicz (use `wandb login --relogin` to force relogin)


env: "WANDB_NOTEBOOK_NAME"="DnCNN Testing"
Using cuda device


In [2]:
sweep_config ={
    'method': 'random'
}

metric = {
    'name': 'loss',
    'goal': 'minimize'
}

sweep_config['metric'] = metric


In [3]:

parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
    },
    'fc_num_layers': {
        'values': [17, 20, 25, 30]
    },
    'fc_num_features': {
        'values': [64, 128, 256]
    }
}

sweep_config['parameters'] = parameters_dict

In [4]:
parameters_dict.update({
    'epochs': {
        'value': 2}
})

In [5]:
parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
    },
    'batch_size': {
        # integers between 16 and 256
        'values': [16, 32, 64, 128, 256]
    }
})

In [6]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'values': [16, 32, 64, 128, 256]},
                'epochs': {'value': 2},
                'fc_num_features': {'values': [64, 128, 256]},
                'fc_num_layers': {'values': [17, 20, 25, 30]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'optimizer': {'values': ['adam', 'sgd']}}}


In [7]:
sweep_id = wandb.sweep(sweep_config, project="pytorch-DnCNN")

Create sweep with ID: hurj02e6
Sweep URL: https://wandb.ai/mdowicz/pytorch-DnCNN/sweeps/hurj02e6


In [8]:
import torch
from torch import nn
from torch.utils.data import DataLoader
# from model import DnCNN
# from Dataset import Img_Dataset
import numpy as np 
import pathlib
import matplotlib.pyplot as plt

# Importing utitility functions for training
from PT_files.model import DnCNN, DnCNN_B
from PT_files.Dataset import Img_Dataset, Large_Img_Dataset
import PT_files.preprocess_data as ppd
import PT_files.save_load as sl

device = "cuda" if torch.cuda.is_available() else "cpu"

In [9]:
training_data = sl.NERSC_load('training_data_60%_6000.npy')
test_data = sl.NERSC_load('test_data_40%_6000.npy')

In [10]:
train_dataset = Img_Dataset(data_set=training_data,
                                  patch_size=150,
                                  width=6000,
                                  height=6000)

test_dataset = Img_Dataset(data_set=test_data,
                                patch_size=150,
                                width=6000,
                                height=6000)

In [11]:
def train_one_step(model, imgs, labels, optimizer):
    optimizer.zero_grad()
  
    
    output = model(imgs)
    loss_fn = nn.MSELoss(reduction='sum')
    loss = loss_fn(output, labels) /(2*len(imgs))

    loss.backward()
    optimizer.step()
    return loss

def train_one_epoch(model, dataloader, optimizer):
    model.train()
    total_loss = 0
    for batch_index, (imgs, labels) in enumerate(dataloader):
        # imgs = imgs.type(torch.FloatTensor)
        # labels = labels.type(torch.FloatTensor)
        
        loss = train_one_step(model, imgs, labels, optimizer)
        total_loss += loss
        
    return np.log(total_loss)

In [12]:
def validate_one_step(model, imgs, labels):
    output = model(imgs)
    loss_fn = nn.MSELoss(reduction='sum')
    loss = loss_fn(output, labels) /(2*len(imgs))
    return loss

def validate_one_epoch(model, dataloader):
    model.eval()
    total_loss = 0
    for batch_index, (imgs, labels) in enumerate(dataloader):
        # imgs = imgs.type(torch.FloatTensor)
        # labels = labels.type(torch.FloatTensor)
        with torch.no_grad():
            loss = validate_one_step(model, imgs, labels)
        total_loss += loss
    return np.log(total_loss)

In [13]:
def build_optimizer(model, optimizer, learning_rate):
    
    if optimizer == "sgd":
        optimizer = torch.optim.SGD(mode.parameters(),
                                    lr=learning_rate,
                                    momentum=0.9)
    elif optimizer == "adam":
        optimizer = torch.optim.Adam(model.parameters(),
                                     lr=learning_rate)
    return optimizer

In [14]:
sweep_config

{'method': 'random',
 'metric': {'name': 'loss', 'goal': 'minimize'},
 'parameters': {'optimizer': {'values': ['adam', 'sgd']},
  'fc_num_layers': {'values': [17, 20, 25, 30]},
  'fc_num_features': {'values': [64, 128, 256]},
  'epochs': {'value': 2},
  'learning_rate': {'distribution': 'uniform', 'min': 0, 'max': 0.1},
  'batch_size': {'values': [16, 32, 64, 128, 256]}}}

In [15]:
def get_dataloader(is_train, batch_size, slice=5):
    "Get a training dataloader"
    full_dataset = torchvision.datasets.MNIST(root=".", train=is_train, transform=T.ToTensor(), download=True)
    sub_dataset = torch.utils.data.Subset(full_dataset, indices=range(0, len(full_dataset), slice))
    loader = torch.utils.data.DataLoader(dataset=sub_dataset, 
                                         batch_size=batch_size, 
                                         shuffle=True if is_train else False, 
                                         pin_memory=True, num_workers=2)
    return loader

In [20]:
def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If calld by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        
        # Training Data
        train_loader = DataLoader(train_dataset, 
                                  config.batch_size)
        # Validation Data
        val_loader = DataLoader(test_dataset,
                                 batch_size=config.batch_size)
        
        # Model
        model = DnCNN(num_layers=config.fc_num_layers,
                        num_features=config.fc_num_features)
        
        # Optimizer
        optimizer = build_optimizer(model, config.optimizer, config.learning_rate)
        
        # Track loss curves
        losses, val_losses = [], []
        
        for epoch in range(config.epochs):
            
            # train_metrics = {"train_loss": train_one_epoch(model, train_loader, optimizer).cpu().detach().numpy()}
            wandb.log({"train_loss": train_one_epoch(model, train_loader, optimizer).cpu().detach().numpy()})
            wandb.log({"val_loss": validate_one_epoch(model, val_loader, optimizer).cpu().detach().numpy()})

            # val_metrics = {"val_loss": validate_one_epoch(model, test_loader).cpu().detach().numpy()}
            # wandb.log({**train_metrics, **val_metrics})

            losses.append(train_one_epoch(model, train_loader, optimizer)).cpu().detach().numpy()
            val_loses.append(validate_one_epoch(model, test_loader)).cpu().detach().numpy()
                             
            
    wandb.finish()

In [21]:
wandb.agent(sweep_id, train, count=5)

wandb: Agent Starting Run: 1y8t00ug with config:
wandb: 	batch_size: 16
wandb: 	epochs: 2
wandb: 	fc_num_features: 256
wandb: 	fc_num_layers: 20
wandb: 	learning_rate: 0.08958099967001429
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Run 1y8t00ug errored: RuntimeError('Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same')
wandb: ERROR Run 1y8t00ug errored: RuntimeError('Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same')
wandb: Agent Starting Run: ssat8esi with config:
wandb: 	batch_size: 128
wandb: 	epochs: 2
wandb: 	fc_num_features: 64
wandb: 	fc_num_layers: 20
wandb: 	learning_rate: 0.09091582147923336
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Run ssat8esi errored: NameError("name 'mode' is not defined")
wandb: ERROR Run ssat8esi errored: NameError("name 'mode' is not defined")
wandb: Agent Starting Run: e68vvmc4 with config:
wandb: 	batch_size: 32
wandb: 	epochs: 2
wandb: 	fc_num_features: 128
wandb: 	fc_num_layers: 17
wandb: 	learning_rate: 0.04435675902446791
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Run e68vvmc4 errored: NameError("name 'mode' is not defined")
wandb: ERROR Run e68vvmc4 errored: NameError("name 'mode' is not defined")
Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true


In [18]:
# wandb.agent(sweep_id, train(train_ds=train_dataset, val_ds=test_dataset, config=sweep_config), count=5)